In [2]:
import pandas as pd
import json
import os


# set option to get rid of column character limit restriction
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

folder = os.chdir(r'C:...Kaggle/2020-03-27/biorxiv_medrxiv/')


filename = [fname for fname in os.listdir(folder) if fname.endswith(".json")]

In [3]:
# find all nested dictionary keys and exract into a list
# Example : 
# sample_dictionary = {1:'a',2:'a',3:{11:'aa',22:'bb',33:{900:'x',901:'y'}},4:'c',5:{111:'aaa',222:'bbb'}}
# find_all_dictionary(sample_dictionary) ---> ['3', '33', '5']

def find_all_dictionary(item):
    dict_list = []

    def recursive_dict_search(item):
        for key, val in item.items():
            if isinstance(val, dict):
                if str(key) not in dict_list:
                    dict_list.append(str(key))
                recursive_dict_search(val)
            elif isinstance(val, list):
                for itr in val:
                    if isinstance(itr, dict):
                        recursive_dict_search(itr)

        return dict_list

    if isinstance(item, dict):
        return recursive_dict_search(item)
    elif isinstance(item, list):
        for element in item:
            if isinstance(element, dict):
                recursive_dict_search(element)
        return dict_list

In [4]:
# returns type of dataframe column as a dictionary
def dataframe_column_type(dataframe):
    column_type = {}
    for col in dataframe.columns:
        column_type[col] = str(type(dataframe[col][0])).replace("<class '",'').replace("'>",'')
    return column_type

In [5]:
# loop all json file
num = 0
for fname in filename[:]:

    num += 1

    # Reading the json as a dict
    with open(fname) as json_data:
        data = json.load(json_data)

    df = pd.DataFrame.from_dict(data, orient='index').T

    # Some columns of df contain so many data that can be converted in dataframe too
    new_dataframes = []
    for key, value in dataframe_column_type(df).items():

        if value == 'dict':
            temp = "df_" + key + " = pd.DataFrame.from_dict(dict(df['" + key + "']), orient='index')"
            exec(temp)
            name_of_dataframe = 'df_' + key
            new_dataframes.append(name_of_dataframe)
        elif value == 'list':
            temp = "df_" + key + " = pd.DataFrame.from_dict(dict(df['" + key + "']))"
            exec(temp)
            temp = "df_" + key + ' = pd.DataFrame.from_dict(dict(df_' + key + ".iloc[:,0]), orient='index')"
            exec(temp)
            name_of_dataframe = 'df_' + key
            new_dataframes.append(name_of_dataframe)

    # First column of df which contains so many data that can be converted in dataframe        
    new_sub_dataframes = []
    for key,val in dataframe_column_type(eval(new_dataframes[0])).items():
        if val == 'list':
            temp = "df_pre_" + key + " = pd.DataFrame.from_dict(dict(" + new_dataframes[0] + "['" + key + "']))"
            new_sub_dataframes.append('df_pre_' + key)
            exec(temp)
            temp = "df_" + key + " = pd.DataFrame.from_dict(dict(" + "df_pre_" + key + ".iloc[:,0]), orient='index')"
            new_sub_dataframes.append('df_' + key)
            exec(temp)

    # Control if new sub dataframes has possibility to be able to converted in dataframe
    sub_fields = []
    for i in eval(new_sub_dataframes[0])[0]:
        sub_fields += find_all_dictionary(i)
    
    sub_fields = list(dict.fromkeys(sub_fields))

    # check if there is more fields that can be converted in dataframe
    if len(sub_fields)>0:

        # next generation dataframe
        dframe_name = []
        for i in sub_fields:
            if sub_fields.index(i) == 0:
                temp = "df_" + i + " = " + new_sub_dataframes[1] + "['" + i + "'].apply(pd.Series)"
                exec(temp)
                dframe_name.append("df_" + i)
            else:
                temp = "df_" + i + " = " + dframe_name[sub_fields.index(i)-1] + "['" + i + "'].apply(pd.Series)"
                exec(temp)
                dframe_name.append("df_" + i)

        dframe_name.insert(0,new_sub_dataframes[1])

        # concatenate dataframe
        my_iter = iter(dframe_name)

        if len(dframe_name)==2:
            temp = "df_final = pd.concat([" + next(my_iter) + ", " + next(my_iter) + "], axis = 1).drop(" + str(sub_fields) + ", axis = 1)"
            exec(temp)
        elif len(dframe_name)==3:
            temp = "df_final = pd.concat([" + next(my_iter) + ", " + next(my_iter) + ", " + next(my_iter) + "], axis = 1).drop(" + str(sub_fields) + ", axis = 1)"
            exec(temp)
        
        df_final.insert(0, 'paper_id', ''.join(df.paper_id))
        df_final.insert(1, 'title', ''.join(df_metadata.title))
        
        if num > 1:
            df_all_researchers = df_all_researchers.append(df_final, sort=False)
        else:
            df_all_researchers = df_final
        
        # clear last df_final dataframe before new loop
        df_final = df_final[0:0]
        
# clean fields from number and dots
df_all_researchers['country'] = df_all_researchers['country'].str.replace('[^a-zA-Z,]','')

# convert list to string
df_all_researchers['middle'] = df_all_researchers.middle.apply(', '.join)

df_all_researchers.to_csv(r'C:...Kaggle/2020-03-13/all_researchers.csv', index=False)

In [9]:
df_all_researchers[90:100]

,paper_id,title,first,middle,last,suffix,email,laboratory,institution,settlement,region,country,0,postCode,addrLine,postBox
0,03ea3a614b56409d3f099c9ad764864293132540,Live-cell single RNA imaging reveals bursts of translational frameshifting,Kenneth,,Lyon,,,,Colorado State University,NaN,CO,USA,NaN,80523,Fort Collins,NaN
1,03ea3a614b56409d3f099c9ad764864293132540,Live-cell single RNA imaging reveals bursts of translational frameshifting,Luis,U,Aguilera,,,,Colorado State University,NaN,CO,USA,NaN,80523,Fort Collins,NaN
2,03ea3a614b56409d3f099c9ad764864293132540,Live-cell single RNA imaging reveals bursts of translational frameshifting,Tatsuya,,Morisaki,,,,Colorado State University,NaN,CO,USA,NaN,80523,Fort Collins,NaN
3,03ea3a614b56409d3f099c9ad764864293132540,Live-cell single RNA imaging reveals bursts of translational frameshifting,Brian,,Munsky,,,,Colorado State University,NaN,CO,USA,NaN,80523,Fort Collins,NaN
4,03ea3a614b56409d3f099c9ad764864293132540,Live-cell single RNA imaging reveals bursts of translational frameshifting,Timothy,J,Stasevich,,tim.stasevich@colostate.edu,,Colorado State University,NaN,CO,USA,NaN,80523,Fort Collins,NaN
0,04030bba3035a58c7725ae267973206f6eb6c0b4,Development and Evaluation of A CRISPR-based Diagnostic For 2019-novel Coronavirus,Tieying,,Hou,,,Laboratory Medicine,Guangdong Academy of Medical Sciences,Guangdong,NaN,China,NaN,510000,7 Guangzhou,NaN
1,04030bba3035a58c7725ae267973206f6eb6c0b4,Development and Evaluation of A CRISPR-based Diagnostic For 2019-novel Coronavirus,Weiqi,,Zeng,,,,Vision Medicals,Guangzhou,Guangdong,China,NaN,510000,NaN,NaN
2,04030bba3035a58c7725ae267973206f6eb6c0b4,Development and Evaluation of A CRISPR-based Diagnostic For 2019-novel Coronavirus,Minling,,Yang,,,,Vision Medicals,Guangzhou,Guangdong,China,NaN,510000,NaN,NaN
3,04030bba3035a58c7725ae267973206f6eb6c0b4,Development and Evaluation of A CRISPR-based Diagnostic For 2019-novel Coronavirus,Wenjing,,Chen,,,,Vision Medicals,Guangzhou,Guangdong,China,NaN,510000,NaN,NaN
4,04030bba3035a58c7725ae267973206f6eb6c0b4,Development and Evaluation of A CRISPR-based Diagnostic For 2019-novel Coronavirus,Lili,,Ren,,,NHC Key Laboratory of Systems Biology of Pathogens and Christophe Mérieux Laboratory,Peking Union Medical College,Beijing,NaN,China,NaN,100730,NaN,NaN
